### Intro

Thanks for checking out my notebook. Here we will shortly explore some bits of usefull insights about the task we are given. Hopefully you find it useful.

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow.keras.layers as L
from tensorflow.keras.models import Sequential
from sklearn.model_selection import train_test_split

%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 5);
sns.set_style('whitegrid')

*Whatis Mechanisms of Action anyway?* Quickly looking at <a href='https://www.cancer.gov/publications/dictionaries/cancer-terms'>NCI Dictionary of Cancer Terms</a> we find the defenition:
> In medicine, a term used to describe how a drug or other substance produces an effect in the body. For example, a drug’s mechanism of action could be how it affects a specific target in a cell, such as an enzyme, or a cell function, such as cell growth.

In [ ]:
root_dir = '../input/lish-moa/'
dfs = {}

for file in os.listdir(root_dir):
    dfs[file.split('.')[0]] = pd.read_csv(os.path.join(root_dir, file))
for k, v in dfs.items():
    print(k)
    print(v.shape)
    print('-'*10)

So we have over 800 features, over 200 labels, over 23k train samples and almost 4k test samples. It is sensible to assume target vectors will be sparse despite problem being multilabel one. But let's not jump to conclusions just yet.

Also here is what we should highlight from data descriptions:

> Features g- signify gene expression data, and c- signify cell viability data. cp_type indicates samples treated with a compound (cp_vehicle) or with a control perturbation (ctrl_vehicle); control perturbations have no MoAs; cp_time and cp_dose indicate treatment duration (24, 48, 72 hours) and dose (high or low). 

In [ ]:
for k in ['test_features', 'train_features']:
    print(k)
    compound_cols = [x for x in dfs[k].keys() if x.startswith('cp_')]
    gene_cols = [x for x in dfs[k].keys() if x.startswith('g-')]
    cell_cols = [x for x in dfs[k].keys() if x.startswith('c-')]
    print('compound features:', len(compound_cols),
         'gene features:', len(gene_cols),
         'cell features:', len(cell_cols))
    """
    uncomment if needed
    print('compounds:')
    print(dfs[k][compound_cols].describe())
    print('genes:')
    print(dfs[k][gene_cols].describe())
    print('cells:')
    print(dfs[k][cell_cols].describe())
    """
    print('~'*10)

In [ ]:
for c in compound_cols:
    print('train:\n', dfs['train_features'][c].value_counts())
    print('test:\n', dfs['test_features'][c].value_counts())

So here we have all features balanced but one: `cp_type`. As we understood before this minority class `ctrl_vehicle` should have zero action for all labels. Take a note as we go.

In [ ]:
fig, ax = plt.subplots(2, 2)
gs = [dfs['train_features'][gene_cols[x]] for x in np.random.choice(list(range(len(gene_cols))), size=4)]
sns.distplot(gs[0], ax=ax[0][0]);
sns.distplot(gs[1], ax=ax[0][1]);
sns.distplot(gs[2], ax=ax[1][0]);
sns.distplot(gs[3], ax=ax[1][1]);
plt.tight_layout();

In [ ]:
fig, ax = plt.subplots(2, 2)
gs = [dfs['train_features'][cell_cols[x]] for x in np.random.choice(list(range(len(cell_cols))), size=4)]
sns.distplot(gs[0], ax=ax[0][0], color='gold');
sns.distplot(gs[1], ax=ax[0][1], color='gold');
sns.distplot(gs[2], ax=ax[1][0], color='gold');
sns.distplot(gs[3], ax=ax[1][1], color='gold');
plt.tight_layout();

Both genes and cells values are normaly distributes except a few. Could those be outliers? Perphaps... 

In [ ]:
for df in dfs.values():
    print(df.isna().sum().sum())

In [ ]:
dfs['train_targets_scored'].head(3)

In [ ]:
dfs['train_targets_scored'].drop('sig_id', axis=1).sum().sort_values()

Well we can see clear some labels are in danger of being overlooked by any model we might build simply because they are underrepresented. Keep that in mind as we progress.

In [ ]:
corr_t = dfs['train_targets_scored'].corr().abs()
sns.heatmap(corr_t[corr_t>0.75], cmap='OrRd');

In [ ]:
temp_df = pd.merge(dfs['train_targets_scored'],
                    dfs['train_features'], on='sig_id')
corr = temp_df.corr()

In [ ]:
corr.dropna(inplace=True)
sns.heatmap(corr, cmap='coolwarm');

We can observe really week correclations between raw features and targets, however, the correlation between the features themselves are quite strong. This might helpful to use dimentionality reduction techniques when we will build up the solution.

In [ ]:
del temp_df

In [ ]:
targets = dfs['train_targets_scored'].drop('sig_id', axis=1).sum()
    
least = sorted(targets.items(), key=lambda x: x[1])[:10]
most = sorted(targets.items(), key=lambda x: x[1])[-10:]
least_x = [i[0] for i in least]
least_y = [i[1] for i in least]
most_x = [i[0] for i in most]
most_y = [i[1] for i in most]
fig, ax = plt.subplots(1, 2)
ax[0].bar(least_x, least_y, color='purple');
ax[0].tick_params(axis='x', labelrotation=75);
ax[1].bar(most_x, most_y, color='purple');
ax[1].tick_params(axis='x', labelrotation=75);

In [ ]:
targets_ns = dfs['train_targets_nonscored'].drop('sig_id', axis=1).sum()
    
least = sorted(targets_ns.items(), key=lambda x: x[1])[:10]
most = sorted(targets_ns.items(), key=lambda x: x[1])[-10:]
least_x = [i[0] for i in least]
least_y = [i[1] for i in least]
most_x = [i[0] for i in most]
most_y = [i[1] for i in most]
fig, ax = plt.subplots(1, 2)
ax[0].bar(least_x, least_y, color='green');
ax[0].tick_params(axis='x', labelrotation=75);
ax[1].bar(most_x, most_y, color='green');
ax[1].tick_params(axis='x', labelrotation=75);

Here is where we find the classes that are not even present, i.e. non of the train samples is labeled as say calpain_inhibitor. The presence of this class actually suggest we will have it in the test dataset. Out of curiosity let's see how much such empty labels we have.

In [ ]:
len(targets_ns[targets_ns==0])

#### Baseline model
To see if we might encounter problems during submission let's create a simple model and make some predictions. Perphaps neural network is too much overkill for you but it allows us to work with highly dimentional data with minimal preprocessing.

In [ ]:
X = dfs['train_features'].drop('sig_id', axis=1)
X['cp_type'] = X['cp_type'].apply(lambda x: 0 if x=='trt_cp' else 1)
X['cp_time'] = X['cp_time'].apply(lambda x: 0 if x==24 else 1)
X['cp_dose'] = X['cp_dose'].apply(lambda x: 0 if x=='D1' else 1)
y = dfs['train_targets_scored'].drop('sig_id', axis=1)

X_train, X_test, y_train, y_test = train_test_split(
    X.values, y.values,
    test_size=0.25)

model = Sequential()
model.add(L.Input(shape=X.shape[1]))
model.add(L.Dense(512, activation='relu'))
model.add(L.Dense(y.shape[1], activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['binary_crossentropy', 'accuracy'])
model.summary()

In [ ]:
model.fit(X_train, y_train, epochs=5)
model.evaluate(X_test, y_test)

In [ ]:
X = dfs['test_features'].drop('sig_id', axis=1)
X['cp_type'] = X['cp_type'].apply(lambda x: 0 if x=='trt_cp' else 1)
X['cp_time'] = X['cp_time'].apply(lambda x: 0 if x==24 else 1)
X['cp_dose'] = X['cp_dose'].apply(lambda x: 0 if x=='D1' else 1)

In [ ]:
preds = model.predict(X)

In [ ]:
sub = dfs['sample_submission']
for i, p in zip(sub.index, preds):
    sub.loc[i, 1:] = p
sub.head(3)

In [ ]:
sub.to_csv('submission.csv', index=False)

I hope some of the highlights will help you get started. Good luck!